# **Limpieza**

#### LIBRERÍAS

In [1]:
import pandas as pd # Manipulación df
import os # Directorios
import re # Manipulación de cadenas de texto (expresiones regulares)
import unicodedata # Función para eliminar acentos
from datetime import datetime, date  # Para trabajar con fechas y horas
from currency_converter import CurrencyConverter # convertidor de monedas
from dateutil.relativedelta import relativedelta  # Para calcular diferencias entre fechas, incluyendo años, meses y días
# Mapear coincidencia
from fuzzywuzzy import process
from fuzzywuzzy import fuzz 
# Instalar python-Levenshtein para mejorar el rendimiento de SequenceMatcher
import Levenshtein

Estilo de impresiones

In [2]:
# Colores ANSI
BLACK = "\033[90m"
RED = "\033[91m"
GREEN = "\033[92m"
YELLOW = "\033[93m"
BLUE = "\033[94m"
MAGENTA = "\033[95m"
CYAN = "\033[96m"
WHITE = "\033[97m"
RESET = "\033[0m"  # Restablecer el color por defecto
# Estilos de texto
BOLD = "\033[1m"
UNDERLINE = "\033[4m"
BLINK = "\033[5m"
REVERSE = "\033[7m"  # Invertir colores (fondo y texto)


#### CARGAR ARCHIVO

Solo cambia el nombre en 'archivo_filename' y su ruta con 'archivo_dir'

In [3]:
directorio_actual = os.getcwd() # Directorio actual de trabajo
# Ruta relativa al archivo CSV
archivo_filename = "ruta/relativa/al/archivo.csv"
archivo_dir = os.path.join(directorio_actual)
archivo_path = os.path.join(archivo_dir, archivo_filename)

In [3]:
directorio_actual = os.getcwd() # Directorio actual de trabajo
# Ruta relativa al archivo CSV
archivo_filename = "C:\\xampp\\htdocs\\datalpine\\resources\\db\\Scrapining\\ciudades\\puebla\\clean\\agosto_2024.csv"
archivo_dir = os.path.join(directorio_actual)
archivo_path = os.path.join(archivo_dir, archivo_filename)

Elige el tipo de archivo a leer

In [4]:
#df = pd.read_excel(archivo_path) 
df = pd.read_csv(archivo_path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3476 entries, 0 to 3475
Data columns (total 30 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         3476 non-null   int64  
 1   categoria                  3235 non-null   object 
 2   precio                     3476 non-null   float64
 3   precio_mxn                 3476 non-null   float64
 4   precio_usd                 3476 non-null   int64  
 5   fecha_conversion           3476 non-null   object 
 6   propiedad                  3476 non-null   object 
 7   metros_total               3476 non-null   int64  
 8   metros_construido          3476 non-null   int64  
 9   precio_m2_terreno          3476 non-null   float64
 10  precio_m2_construido       3476 non-null   float64
 11  tiempo_de_publicacion      3476 non-null   object 
 12  meses_transcurridos        3476 non-null   int64  
 13  meses_transcurridos_fecha  3476 non-null   objec

#### LIMPIEZA NULOS

In [6]:
df.fillna({'precio': 0}, inplace=True) 
df.fillna({'denominacion': ''}, inplace=True) 
df.fillna({'propiedad': 'Propiedad sin título'}, inplace=True) 
df.fillna({'metros_total': 0}, inplace=True) 
df.fillna({'metros_construido': 0}, inplace=True) 
#df.fillna({'tiempo_de_publicacion': ''}, inplace=True) 
df.fillna({'tipo': 'Propiedad sin tipo de casa'}, inplace=True) 
df.fillna({'estacionamientos': 0}, inplace=True) 
df.fillna({'recamaras': 0}, inplace=True) 
df.fillna({'banos': 0}, inplace=True) 
df.fillna({'medio_banos': 0}, inplace=True) 
df.fillna({'seguridad_privada': 'No'}, inplace=True) 
df.fillna({'ubicacion': 'Propiedad sin ubicación'}, inplace=True) 
df.fillna({'url': '-'}, inplace=True) 
df.fillna({'descripcion': "Propiedad sin descripcion"}, inplace=True) 

# Duplicados
* Estandarizar descripción
* Eliminación de los duplicados, dejando uno de los resultantes

In [107]:
df.columns

Index(['precio', 'denominacion', 'propiedad', 'metros_total',
       'metros_construido', 'tiempo_de_publicacion', 'tipo',
       'estacionamientos', 'recamaras', 'banos', 'medio_banos',
       'seguridad_privada', 'lugar', 'Colonia', 'CP', 'ubicacion', 'url',
       'descripcion'],
      dtype='object')

In [16]:
# Convierte todos los valores a strings, maneja NaN o None como cadenas vacías
df['descripcion'] = df['descripcion'].astype(str)
# Convertir a minúsculas
df['descripcion'] = df['descripcion'].str.lower()
# Eliminar acentos y carácteres especiales
import unicodedata 
def eliminar_acentos(texto):
    texto_normalizado = unicodedata.normalize('NFKD', texto)
    return ''.join(char for char in texto_normalizado if unicodedata.category(char) != 'Mn')
df['descripcion'] = df['descripcion'].apply(eliminar_acentos)
df['descripcion'] = df['descripcion'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)
inicio = df.shape

## a) Con todos los datos iguales

In [109]:
duplicados = df.loc[df.duplicated()]
shapes = duplicados.shape
print(duplicados)
print(f'{BLUE}Los duplicados encontrados son: {RESET}',f'{RED}{shapes}{RESET}')

        precio denominacion  \
24      372608          MXN   
27      373340          MXN   
29      373500          MXN   
33      395100          MXN   
41      400012          MXN   
...        ...          ...   
4440  31674000          MXN   
4441  31674000          MXN   
4443  32000000          MXN   
4445  33000000          MXN   
4450  35000000          MXN   

                                                                                    propiedad  \
24    ¡¡EXCELENTE OPORTUNIDAD DE INVERSIÒN!! HERMOSA CASA EN VENTA. EL CERRITO, PUEBLA PUEBLA   
27                                     CASA EN VENTA EN EL FRACCIONAMIENTO SANTA CLARA PUEBLA   
29                                                  Casa en venta en San José, Puebla, México   
33                                                 VENTA DE CASA EN COLONIA LOMA LINDA PUEBLA   
41                            CASA EN VENTA. ESPAÑA COL GEOVILLAS DEL SUR. PUEBLA, PUEBLA. BM   
...                                          

In [110]:
df = df.drop_duplicates(keep='first')
shapes = df.shape
print('La base mantuvo una estructura de: ',f'{RED}{shapes}{RESET}')

La base mantuvo una estructura de:  (3712, 18)


In [111]:
duplicados = df.loc[df.duplicated()]
print(duplicados)

Empty DataFrame
Columns: [precio, denominacion, propiedad, metros_total, metros_construido, tiempo_de_publicacion, tipo, estacionamientos, recamaras, banos, medio_banos, seguridad_privada, lugar, Colonia, CP, ubicacion, url, descripcion]
Index: []


In [112]:
shapes = df.shape
print('La base ha quedado con: ',f'{RED}{shapes}{RESET}', 'de',f'{GREEN}{inicio}{RESET}')

La base ha quedado con:  (3712, 18) de (4454, 18)


## b) Descripción y precio iguales

In [30]:
duplicados = df.loc[df.duplicated(subset=['descripcion','precio'])]
shapes = duplicados.shape
print(duplicados)
print(f'{BLUE}Los duplicados encontrados son: {RESET}',f'{RED}{shapes}{RESET}')

          precio denominacion  \
71      442000.0          MXN   
77      450000.0          MXN   
142     504000.0          MXN   
180     519000.0          MXN   
199     528521.0          MXN   
...          ...          ...   
2874   5790000.0          MXN   
3099   8900000.0          MXN   
3157   9960000.0          MXN   
3251  12250000.0          MXN   
3407  19900000.0          MXN   

                                              propiedad  metros_total  \
71      VENTA HERMOSA CASA EN PLAZAS AMALUCAN EN PUEBLA           168   
77          Casa en venta en colonia Cuauhtémoc, Puebla           150   
142               Casa en venta en Valle del Sol Puebla           180   
180                 VENTA DE CASA EN PUEBLA DE ZARAGOZA           211   
199                             VENTA DE CASA EN PUEBLA           224   
...                                                 ...           ...   
2874                   Casa en Venta, Colonia La Calera           375   
3099            Cas

In [31]:
df = df.drop_duplicates(subset=['descripcion','precio'],keep='first')
shapes = df.shape
print('La base mantuvo una estructura de: ',f'{RED}{shapes}{RESET}')

La base mantuvo una estructura de:  (3398, 15)


In [32]:
duplicados = df.loc[df.duplicated(subset=['descripcion','precio'])]
print(duplicados)

Empty DataFrame
Columns: [precio, denominacion, propiedad, metros_total, metros_construido, tiempo_de_publicacion, tipo, estacionamientos, recamaras, banos, medio_banos, seguridad_privada, ubicacion, url, descripcion]
Index: []


## c) M2 construcción, Ubicación y Precio

In [33]:
duplicados = df.loc[df.duplicated(subset=['metros_construido','ubicacion','precio'])]
shapes = duplicados.shape
print(duplicados)
print(f'{BLUE}Los duplicados encontrados son: {RESET}',f'{RED}{shapes}{RESET}')

          precio denominacion  \
14      356000.0          MXN   
15      356000.0          MXN   
27      372608.0          MXN   
33      382900.0          MXN   
43      400000.0          MXN   
...          ...          ...   
3374  17000000.0          MXN   
3384  17900000.0          MXN   
3425  22000000.0          MXN   
3430  24800000.0          MXN   
3466  38000000.0          MXN   

                                              propiedad  metros_total  \
14                              CASA EN VENTA EN PUEBLA            80   
15                 EXCELENTE CASA OPORTUNIDAD INVERSION            70   
27    ¡¡EXCELENTE OPORTUNIDAD DE INVERSIÒN!! HERMOSA...           120   
33                      VENTA DE HERMOSA CASA EN PUEBLA           150   
43                              VENTA DE CASA EN PUEBLA           150   
...                                                 ...           ...   
3374  CASA EN VENTA FRACC. CAMPESTRE DEL BOSQUE, EST...           700   
3384  Residencia Ex

In [34]:
df = df.drop_duplicates(subset=['metros_construido','ubicacion','precio'],keep='first')
shapes = df.shape
print('La base mantuvo una estructura de: ',f'{RED}{shapes}{RESET}')

La base mantuvo una estructura de:  (2958, 15)


In [35]:
duplicados = df.loc[df.duplicated(subset=['metros_construido','ubicacion','precio'])]
print(duplicados)

Empty DataFrame
Columns: [precio, denominacion, propiedad, metros_total, metros_construido, tiempo_de_publicacion, tipo, estacionamientos, recamaras, banos, medio_banos, seguridad_privada, ubicacion, url, descripcion]
Index: []


## d) Características y Precio

In [36]:
duplicados = df.loc[df.duplicated(subset=['estacionamientos','recamaras','banos','medio_banos','precio'])]
shapes = duplicados.shape
print(duplicados)
print(f'{BLUE}Los duplicados encontrados son: {RESET}',f'{RED}{shapes}{RESET}')

          precio denominacion  \
6       325000.0          MXN   
13      356000.0          MXN   
16      356000.0          MXN   
31      382000.0          MXN   
35      383251.0          MXN   
...          ...          ...   
3373  17000000.0          MXN   
3413  20000000.0          MXN   
3419  21000000.0          MXN   
3433  25000000.0          MXN   
3434  25000000.0          MXN   

                                              propiedad  metros_total  \
6                    BELLISIMA CASA EN MAYORAZGO PUEBLA            85   
13                               VENTA DE CASA EN PUBLA            80   
16    BONITA CASA EN VENTA EN RESIDENCIAL SANTA CLAR...            80   
31           HERMOSA CASA EN HEROES DE PUEBLA EN PUEBLA           150   
35     VENTA DE CASA EN GRANJAS MAYORAZGO PUEBLA PUEBLA           130   
...                                                 ...           ...   
3373      CASA EN VENTA UBICADA EN CAMPESTRE DEL BOSQUE           700   
3413            Her

In [37]:
df = df.drop_duplicates(subset=['estacionamientos','recamaras','banos','medio_banos','precio'],keep='first')
shapes = df.shape
print('La base mantuvo una estructura de: ',f'{RED}{shapes}{RESET}')

La base mantuvo una estructura de:  (2452, 15)


In [38]:
duplicados = df.loc[df.duplicated(subset=['estacionamientos','recamaras','banos','medio_banos','precio'])]
print(duplicados)

Empty DataFrame
Columns: [precio, denominacion, propiedad, metros_total, metros_construido, tiempo_de_publicacion, tipo, estacionamientos, recamaras, banos, medio_banos, seguridad_privada, ubicacion, url, descripcion]
Index: []


In [40]:
shapes = df.shape
print('La base ha quedado con: ',f'{RED}{shapes}{RESET}', 'de',f'{GREEN}{inicio}{RESET}')

La base ha quedado con:  (2452, 15) de (3477, 15)


# 1. Manipulación de precios

## a) Precio en descripción

Agrega las diferentes formas de que el precio se encuentre en la descripción.

In [113]:
def extraer_precio(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones
    precios = re.findall(r'\$?\s*(\d{1,3}(?:[.,]?\d{3})*(?:[.,]\d{2})?)\s*(mxn|usd|pesos|monto|precio|)', descripcion)
    precio = None
    for match in precios:
        numero, moneda = match
        numero = numero.replace(',', '').replace("'", '')#.replace(".",'')
        try:
            precio = float(numero)
        except ValueError:
            continue
        if precio:
            break
    return precio

# Denominación de la descripción
def extraer_denominacion(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones 
    denominaciones = re.findall(r'\b(mil|mxn|precio|pesos|monto|Mexicanos|Estadounidenses)\b', descripcion)
    return denominaciones[0] if denominaciones else None

df['precio_desc'] = df['descripcion'].apply(extraer_precio)
#df['denominacion_desc'] = df['descripcion'].apply(extraer_denominacion)
inicio = df.shape

In [91]:
# Aplicar colores a los valores de precio y precio_desc
def colorize_row(row):
    row_colored = row.copy()  # Crear una copia para no modificar el original
    row_colored['precio'] = f"{MAGENTA}{row['precio']}{RESET}"
    row_colored['precio_desc'] = f"{GREEN}{row['precio_desc']}{RESET}"
    return row_colored

### Precio [0 / 0]

In [76]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['precio'] == 0 ) & (df['precio_desc'].isna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas
# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    #print(row[['precio', 'precio_desc', 'url']])
    print(colorize_row(row)[['precio', 'precio_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    print("3. Sustituir el valor manualmente")    
    # Solicitar la opción al usuario
    option = input("Elige una opción (1 o 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'precio': ")
        df.at[idx, 'precio'] = float(new_value)
    elif option == '1':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    print("\n---\n")
print("Proceso completado.")

Proceso completado.


Eliminar registros que no recuperaron su precio ni desde la descripción

In [14]:
df = df[~( (df['precio'] == 0 ) & (df['precio_desc'].isna()) )] 

### Precio [0 / #]

In [15]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['precio'] == 0 ) & (df['precio_desc'].notna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    #print(row[['precio', 'precio_desc', 'url']])
    print(colorize_row(row)[['precio', 'precio_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    print("2. Sustituir el valor de 'precio' por el de 'precio_desc'")
    print("3. Sustituir el valor manualmente")
    
    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'precio': ")
        df.at[idx, 'precio'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'precio'] = df.at[idx, 'precio_desc']
    elif option == '1':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")

Proceso completado.


### Precio [# / #]

In [16]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['precio']!= 0) & (df['precio_desc'].notna())) & (df['precio'] != df['precio_desc']) )] 
# Filtro dentro del estándar
totales = totales[(totales['precio_desc'] >= 250000) & (totales['precio_desc'] <= 500000000)]
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    #print(row[['precio', 'precio_desc', 'url']])
    print(colorize_row(row)[['precio', 'precio_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    print("2. Sustituir el valor de 'precio' por el de 'precio_desc'")
    print("3. Sustituir el valor manualmente")
    
    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'precio': ")
        df.at[idx, 'precio'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'precio'] = df.at[idx, 'precio_desc']
    elif option == '1':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")

    print("\n---\n")

print("Proceso completado.")

Registro 22:
precio                                                                       365000.0
precio_desc                                                                  290000.0
url            https://www.lamudi.com.mx/detalle/41032-73-bcd23d0cc1f2-a5f0-2a62b82f-ae26-4ee5
Name: 22, dtype: object

¿Qué acción te gustaría realizar?
1. Pasar al siguiente registro
2. Sustituir el valor de 'precio' por el de 'precio_desc'
3. Sustituir el valor manualmente
Opción no válida. Pasando al siguiente registro...

---

Registro 45:
precio                                                                       402000.0
precio_desc                                                                  250000.0
url            https://www.lamudi.com.mx/detalle/41032-73-9301c25edeab-9356-573dce61-852c-4906
Name: 45, dtype: object

¿Qué acción te gustaría realizar?
1. Pasar al siguiente registro
2. Sustituir el valor de 'precio' por el de 'precio_desc'
3. Sustituir el valor manualmente
Opción no válida. P

In [77]:
shapes = df.shape
print('La base ha quedado con: ',f'{RED}{shapes}{RESET}', 'de',f'{GREEN}{inicio}{RESET}')

La base ha quedado con:  (4702, 16) de (4702, 16)


## b) Denominación MXN

SI EXISTE UNA DENOMINACIÓN QUE NO SEA MXN, SE CONVIERTA EL PRECIO

In [114]:
import datetime
c = CurrencyConverter() # Instancia del convertidor de monedas
def convertir_a_mxn(precio, denominacion):
    if denominacion != 'MXN':
        return int(round(c.convert(precio, denominacion, 'MXN')))
    return precio

def convertir_a_usd(precio, denominacion):
    if denominacion != 'USD':
        return round(c.convert(precio, denominacion, 'USD'))
    return precio

# Obtener la fecha actual
fecha_conversion = datetime.datetime.now().date()

# Aplicar la conversión a cada fila del DataFrame y agregar la fecha de conversión
df['precio_mxn'] = df.apply(lambda row: convertir_a_mxn(row['precio'], row['denominacion']), axis=1)
df['precio_usd'] = df.apply(lambda row: convertir_a_usd(row['precio'], row['denominacion']), axis=1)
df['fecha_conversion'] = fecha_conversion
print(df[['precio', 'denominacion', 'precio_mxn', 'precio_usd', 'fecha_conversion']].head())

   precio denominacion  precio_mxn  precio_usd fecha_conversion
0  280000          MXN      280000       15575       2024-10-04
1  315000          MXN      315000       17522       2024-10-04
2  315000          MXN      315000       17522       2024-10-04
3  315000          MXN      315000       17522       2024-10-04
4  320000          MXN      320000       17800       2024-10-04


## c) Precio fuera del estandar

IDENTIFICAR Y ELIMINAR PROPIEDADES QUE CUENTEN CON PRECIOS BAJOS y ALTOS AL ESTANDAR

In [165]:
inicio = df.shape
# Eliminar los registros donde el precio es menor al estandar (250,000) y con precio_desc vacío
df = df[(df['precio'] >= 250000)] #& (df['precio_desc'].notna())]
# Eliminar los registros donde el precio es mayor al estandar (500,000,000) y con precio_desc vacío
df = df[(df['precio'] <= 500000000)] #& (df['precio_desc'].notna())] 
# Ordenar valores de manera ascendente
print(df['precio'].sort_values())
shapes = df.shape
print('La base ha quedado con: ',f'{RED}{shapes}{RESET}', 'de',f'{GREEN}{inicio}{RESET}')

2          280000
3          280000
4          325000
5          334608
6          344966
          ...    
3627     40000000
3628     40000000
3629     41000000
3630     60000000
3631    149922400
Name: precio, Length: 3584, dtype: int64
La base ha quedado con:  (3584, 19) de (3586, 19)


## D) Segmento o categoría

In [6]:
# Crear un diccionario donde las llaves son las categorías y los valores son las tuplas de rangos
rangos_precio = {
    "E1": (0, 500000),
    "E2": (500000, 750000),
    "E3": (750001, 1000000),
    "D1": (1000001, 1250000),
    "D2": (1250001, 1500000),
    "D3": (1500001, 1750000),
    "C1": (1750001, 2000000),
    "C2": (2000001, 2250000),
    "C3": (2250001, 2500000),
    "B1": (2500001, 2750000),  
    "B2": (2750001, 3000000),  
    "B3": (3000001, 3250000),  
    "A1": (3250001, 3500000),
    "A2": (3500001, 3750000),
    "A3": (3750001, 4000000),
    "S1": (4000001, 6000000),
    "S2": (6000001, 8000000),
    "S3": (8000001, 12000000),
    "L1": (12000001, 14000000),
    "L2": (14000001, 16000000),
    "L3": (16000001, 18000000),
    "L+": (18000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Función para asignar la categoría según el precio
def asignar_categoria(precio):
    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
        if limite_inferior <= precio < limite_superior:
            return categoria
    return None  # En caso de que el precio no caiga en ningún rango (caso raro)

# Asignar la categoría a cada registro
df['categoria'] = df['precio'].apply(asignar_categoria)
print(df['categoria'].unique())

['S3' 'S1' 'E2' 'S2' None 'E3' 'B3' 'A3' 'C3' 'D1' 'B2' 'A1' 'D2' 'C2'
 'A2' 'B1' 'L2' 'C1' 'D3']


# 2. Meses transcurridos

Sacar el número de meses transcurridos desde la fecha de su publicación a la actual

In [5]:
from datetime import datetime
# Convertir el mes abreviado en número
def month_to_int(month):
    months = ['ene', 'feb', 'mar', 'abr', 'may', 'jun', 'jul', 'ago', 'sep', 'oct', 'nov', 'dic']
    return months.index(month.lower()) + 1

# Estandarizar el formato de fecha
def standardize_date(date_str):
    # Caso 1: formato "20 abr. 2023" -> "2023-04-20"
    try:
        day, month, year = date_str.split(' ')
        month_int = month_to_int(month.replace('.', ''))  # Eliminar puntos si existen
        return f"{year}-{month_int:02d}-{int(day):02d}"
    except (ValueError, AttributeError):
        pass

    # Caso 2: formato "YYYY-MM-DD HH:MM:SS" o "YYYY-MM-DD"
    try:
        return pd.to_datetime(date_str).strftime('%Y-%m-%d')
    except (ValueError, TypeError):
        # Si no se puede convertir, devolver None
        return None

# Aplicar la función a la columna 'tiempo_de_publicacion'
df['fecha_estandarizada'] = df['tiempo_de_publicacion'].apply(standardize_date)

# Convertir a datetime y manejar errores
df['fecha_estandarizada'] = pd.to_datetime(df['fecha_estandarizada'], errors='coerce')

# Obtener la fecha actual
#fecha_actual = datetime.now().date()
fecha_actual = datetime.strptime("30/04/24", "%d/%m/%y").date()

# Función para calcular la diferencia en meses
def calcular_meses(fecha):
    if pd.isnull(fecha):
        return None
    rd = relativedelta(fecha_actual, fecha.date())
    return rd.years * 12 + rd.months

# Aplicar la función para calcular los meses transcurridos
df['meses_transcurridos'] = df['fecha_estandarizada'].apply(calcular_meses)

#Registro de fecha de conversión
df['meses_transcurridos_fecha'] = fecha_actual

# Mostrar el DataFrame con las fechas estandarizadas y meses transcurridos
print(df[['tiempo_de_publicacion', 'fecha_estandarizada', 'meses_transcurridos','meses_transcurridos_fecha']])

    tiempo_de_publicacion fecha_estandarizada  meses_transcurridos  \
0              2023-03-29          2023-03-29                   13   
1              2023-04-28          2023-04-28                   12   
2              2024-01-11          2024-01-11                    3   
3              2023-09-13          2023-09-13                    7   
4              2024-02-21          2024-02-21                    2   
..                    ...                 ...                  ...   
127            2024-03-04          2024-03-04                    1   
128            2024-03-04          2024-03-04                    1   
129            2024-02-17          2024-02-17                    2   
130            2023-09-20          2023-09-20                    7   
131            2024-03-04          2024-03-04                    1   

    meses_transcurridos_fecha  
0                  2024-04-30  
1                  2024-04-30  
2                  2024-04-30  
3                  2024-04-30  

In [27]:
fecha_actual

datetime.date(2024, 2, 25)

# 3. Tipo de propiedad

## a) Estandarizar

In [117]:
# Convierte todos los valores a strings, maneja NaN o None como cadenas vacías
df['tipo'] = df['tipo'].astype(str)
# Convertir a minúsculas
df['tipo'] = df['tipo'].str.lower()

import unicodedata# Función para eliminar acentos
def eliminar_acentos(texto):
    # Normaliza y elimina los acentos
    texto_normalizado = unicodedata.normalize('NFKD', texto)
    return ''.join(char for char in texto_normalizado if unicodedata.category(char) != 'Mn')
df['tipo'] = df['tipo'].apply(eliminar_acentos)
# Convertir la primera letra de cada palabra a mayúscula
df['tipo'] = df['tipo'].str.title()
df['tipo'].unique() 

array(['Casa', 'Departamento'], dtype=object)

## b) Lista de tipos deseados

In [118]:
# Filtrar los registros según los tipos especificados
tipos_permitidos = 'Casa|Casa En Condominio|Casa En Fraccionamiento|Departamento|Penthouse|Dúplex|Loft|Estudio|Condominio Horizontal'
df = df[df['tipo'].str.contains(tipos_permitidos, na=False)]
df['tipo'].unique()

array(['Casa', 'Departamento'], dtype=object)

* Casas: Casa en condominio, 
* Departamento: Penthouse, Loft

## c) Manejo de terrenos

Se buscan los registros que no sean las propiedades si no un solo terreno

In [88]:
# Filtrar los registros según las condiciones dadas
totales =df[df['descripcion'].str.contains('lote', case=False)]
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# MANEJAR PROCESO DE ELIMINACIÓN 
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['descripcion']])
    print(row[['url']])
    print(f"{GREEN}\n¿Te gustaría eliminar el registro?{RESET}")
    # Solicitar la opción al usuario
    option = input("Elige una opción: Si(1) / No(0)): ").strip().lower()
    
    if option == '1':
        df.drop(idx, inplace=True)
        print(f"Registro eliminado.\n")
    else:
        print(f"Registro conservado.\n")
        print("\n---\n")
print("Proceso completado.")

Registro 455:
descripcion    oportunidad inmueble adjudicadoya puedes adquirir el hogar de tus suenos a un excelente precio y de forma seguraeste inmueble es un remate bancario lo que significa que puedes obtenerlo muy por abajo de su valor comercial la propiedad se encuentra ubicada en calle 29 sur condominio hacienda santa clara col san antonio cacalotepec puebla puebla importante no se acepta ningun tipo de creditopago de contado en dos exhibiciones1er pago 320000 a la firma de contrato y 2do pago lo realiza a la entrega de la cesion de derechos ante notario publico agenda tu cita para darte detalles del proceso de adquisicion con gusto te atendere araceli casas tel ver telefono whatsapp email contactar en grupo antham nos especializamos en remates bancarios te asesoramos desde el inicio y hasta la entrega de la propiedad ref gaci
Name: 455, dtype: object
url    https://www.lamudi.com.mx//detalle/41032-73-bf77e7bbff24-93db-c4031523-a425-4ba2
Name: 455, dtype: object

¿Te gustaría el

In [89]:
shapes = df.shape
print('La base ha quedado con: ',f'{RED}{shapes}{RESET}', 'de',f'{GREEN}{inicio}{RESET}')

La base ha quedado con:  (3712, 26) de (3712, 19)


# 4. Metros totales (terreno)

## a) Metros en descripción

In [119]:
# Función para extraer totales de la descripción
def extraer_totales(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones de totales
    totales = re.findall(r'\b(\d+[\.,]?\d*)\s*(metros|totales|mt2)\b', descripcion)
    total = None
    for match in totales:
        numero, _ = match
        numero = numero.replace(',', '').replace("'", '')
        try:
            total = float(numero)
        except ValueError:
            continue
        if total:
            break
    return total

# Aplicar la función al DataFrame
df['totales_desc'] = df['descripcion'].apply(extraer_totales)
inicio = df.shape

In [33]:
# Aplicar colores a los valores de precio y precio_desc
def colorize_row(row):
    row_colored = row.copy()  # Crear una copia para no modificar el original
    row_colored['metros_total'] = f"{MAGENTA}{row['metros_total']}{RESET}"
    row_colored['totales_desc'] = f"{GREEN}{row['totales_desc']}{RESET}"
    return row_colored

### Metros [0 / 0]

In [34]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['metros_total'] == 0 ) & (df['totales_desc'].isna() | df['totales_desc']==0) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    #print(row[['metros_total', 'totales_desc', 'url']])
    print(colorize_row(row)[['metros_total', 'totales_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    print("2. Sustituir el valor de 'metros_total' por el de 'totales_desc'")
    print("3. Sustituir el valor manualmente")
    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'metros_total': ")
        df.at[idx, 'metros_total'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'metros_total'] = df.at[idx, 'totales_desc']
    elif option == '1':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")

Proceso completado.


### Metros [0 / #]

In [35]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['metros_total'] == 0 ) & (df['totales_desc'].notna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    #print(row[['metros_total', 'totales_desc', 'url']])
    print(colorize_row(row)[['metros_total', 'totales_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    print("2. Sustituir el valor de 'metros_total' por el de 'totales_desc'")
    print("3. Sustituir el valor manualmente")
    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'metros_total': ")
        df.at[idx, 'metros_total'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'metros_total'] = df.at[idx, 'totales_desc']
    elif option == '1':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    print("\n---\n")

print("Proceso completado.")

Proceso completado.


### Metros [# / #]

In [36]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['metros_total']!= 0) & (df['totales_desc'].notna())) & (df['metros_total'] != df['totales_desc']) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")    
    print(colorize_row(row)[['metros_total', 'totales_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    print("2. Sustituir el valor de 'metros_total' por el de 'totales_desc'")
    print("3. Sustituir el valor manualmente")
    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        while True:
            try:
                new_value = float(input("Introduce el nuevo valor para 'metros_total': "))
                df.at[idx, 'metros_total'] = new_value
                break
            except ValueError:
                print("Por favor, introduce un número válido (entero o decimal).")
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'metros_total'] = df.at[idx, 'totales_desc']
    elif option == '1':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    print("\n---\n")

print("Proceso completado.")

Registro 62:
metros_total                                                                       110
totales_desc                                                                     400.0
url             https://www.lamudi.com.mx/detalle/41032-73-68dbaf81f582-dbca-46fc9d62-a36e-4b31
Name: 62, dtype: object

¿Qué acción te gustaría realizar?
1. Pasar al siguiente registro
2. Sustituir el valor de 'metros_total' por el de 'totales_desc'
3. Sustituir el valor manualmente
Registro 169:
metros_total                                                                      249
totales_desc                                                                    140.0
url             https://www.lamudi.com.mx/detalle/41032-73-e38de706dc5d-42c1-31986f9-90b0-1fc3
Name: 169, dtype: object

¿Qué acción te gustaría realizar?
1. Pasar al siguiente registro
2. Sustituir el valor de 'metros_total' por el de 'totales_desc'
3. Sustituir el valor manualmente
Registro 311:
metros_total                               

In [37]:
shapes = df.shape
print('La base ha quedado con: ',f'{RED}{shapes}{RESET}', 'de',f'{GREEN}{inicio}{RESET}')

La base ha quedado con:  (3476, 24) de (3476, 24)


## b) Manejo por rangos de UMA

Paso opcional por si no se espera cambiar el precio por el que se tenga en descripción

Rangos
* 118 UMAS = 40m2
* 118.1-200 = 50m2
* 200.1-350 = 71m2
* 350.1-750 = 102m2
* 750.1-1500 = 156m2

### DF cuando ambas columnas son 0 o vacíos

In [ ]:
# Rangos de UMAs y sus equivalencias en m²
umas_to_m2 = [
    (118, 40),
    (200, 50),
    (350, 71),
    (750, 102),
    (1500, 156)
]
valor_uma = 108.57

# Convertir MXN a UMAs
def precio_a_umas(precio, valor_uma):
    return precio / valor_uma

# Asignar m² basado en el precio en UMAs
def asignar_m2(precio_umas):
    for upper_bound, m2 in umas_to_m2:
        if precio_umas <= upper_bound:
            return m2
    return None  # En caso de que el precio en UMAs sea mayor al máximo definido

# Actualizar los registros con 'metros_total' igual a 0 y 'totales_desc' es NaN
def actualizar_metros_total(row):
    if row['metros_total'] == 0 and pd.isna(row['totales_desc']):
        precio_umas = precio_a_umas(row['precio'], valor_uma)
        row['metros_total'] = asignar_m2(precio_umas)
    return row

totales = df[( (df['metros_total'] == 0 ) & (df['totales_desc'].isna() | df['totales_desc']==0) )] 
totales = df.apply(actualizar_metros_total, axis=1)
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

#df = df.apply(actualizar_metros_total, axis=1)
print(totales[['metros_total', 'totales_desc', 'url']])

In [ ]:
# Si se aceptan los cambios:
df = df.apply(actualizar_metros_total, axis=1)

# 5. M2 construidos

## a) M-construidos en descripción

In [120]:
# Función para extraer totales de la descripción
def extraer_totales(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones de totales
    totales = re.findall(r'\b(\d+[\.,]?\d*)\s*(metros|construidos)\b', descripcion)
    total = None
    for match in totales:
        numero, _ = match
        numero = numero.replace(',', '').replace("'", '')
        try:
            total = float(numero)
        except ValueError:
            continue
        if total:
            break
    return total

# Aplicar la función al DataFrame
df['construidos_desc'] = df['descripcion'].apply(extraer_totales)
inicio = df.shape

In [84]:
# Aplicar colores a los valores de precio y precio_desc
def colorize_row(row):
    row_colored = row.copy()  # Crear una copia para no modificar el original
    row_colored['metros_construido'] = f"{MAGENTA}{row['metros_construido']}{RESET}"
    row_colored['construidos_desc'] = f"{GREEN}{row['construidos_desc']}{RESET}"
    return row_colored

### Metros [0 / 0]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['metros_construido'] == 0 ) & (df['construidos_desc'].isna() | df['construidos_desc']==0) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    #print(row[['metros_construido', 'construidos_desc', 'url']])
    print(colorize_row(row)[['metros_construido', 'construidos_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    print("2. Sustituir el valor de 'metros_construido' por el de 'construidos_desc'")
    print("3. Sustituir el valor manualmente")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'metros_construido': ")
        df.at[idx, 'metros_construido'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_construido' por 'construidos_desc'
        df.at[idx, 'metros_construido'] = df.at[idx, 'construidos_desc']
    elif option == '1':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")

Proceso completado.


### Metros [0 / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['metros_construido'] == 0 ) & (df['construidos_desc'].notna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    #print(row[['metros_construido', 'construidos_desc', 'url']])
    print(colorize_row(row)[['metros_construido', 'construidos_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    print("2. Sustituir el valor de 'metros_construido' por el de 'construidos_desc'")
    print("3. Sustituir el valor manualmente")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'metros_construido': ")
        df.at[idx, 'metros_construido'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_construido' por 'construidos_desc'
        df.at[idx, 'metros_construido'] = df.at[idx, 'construidos_desc']
    elif option == '1':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")

Proceso completado.


### Metros [# / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['metros_construido']!= 0) & (df['construidos_desc'].notna())) & (df['metros_construido'] != df['construidos_desc']) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    #print(row[['metros_construido', 'construidos_desc', 'url']])
    print(colorize_row(row)[['metros_construido', 'construidos_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    print("2. Sustituir el valor de 'metros_construido' por el de 'construidos_desc'")
    print("3. Sustituir el valor manualmente")
    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'metros_construido': ")
        try:
            df.at[idx, 'metros_construido'] = float(new_value)
        except ValueError:
            print("Valor no válido. Debe ser un número. Pasando al siguiente registro...")
    elif option == '2':
        # Sustituir el valor de 'metros_construido' por 'construidos_desc'
        df.at[idx, 'metros_construido'] = df.at[idx, 'construidos_desc']
    elif option == '1':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


In [ ]:
shapes = df.shape
print('La base ha quedado con: ',f'{RED}{shapes}{RESET}', 'de',f'{GREEN}{inicio}{RESET}')

# 6. Precio / m2

## M2 Totales (terreno)

In [121]:
# Calcular precio por metro cuadrado del terreno
df['precio_m2_terreno'] = df['precio']/df['metros_total']

## M2 Construidos

In [122]:
# Calcular precio por metro cuadrado de la cosntrucción
df['precio_m2_construido'] = df['precio']/df['metros_construido']

# 7. Estacionamientos

## a) Estacionamientos descritos

Asigna las diferentes maneras en las que puede encontrarse descrito el espacio de estacionamientos

In [123]:
# Función para extraer estacionamientos de la descripción
def extraer_coches(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones de totales
    totales = re.findall(r'\b(\d+[\.,]?\d*)\s*(|cochera|garage|carros|carro|coche|coches|autos|automóviles|automóvil|auto|lugares de estacionamiento)\b', descripcion)
    total = None
    for match in totales:
        numero, _ = match
        numero = numero.replace(',', '').replace("'", '').replace('.','')
        try:
            total = float(numero)
        except ValueError:
            continue
        if total:
            break
    return total

# Aplicar la función al DataFrame
df['estacionamiento_desc'] = df['descripcion'].apply(extraer_coches)
inicio = df.shape

In [42]:
# Aplicar colores a los valores de precio y precio_desc
def colorize_row(row):
    row_colored = row.copy()  # Crear una copia para no modificar el original
    row_colored['estacionamientos'] = f"{MAGENTA}{row['estacionamientos']}{RESET}"
    row_colored['estacionamiento_desc'] = f"{GREEN}{row['estacionamiento_desc']}{RESET}"
    return row_colored

### Estacionamientos [0 / 0]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['estacionamientos']==0) & (df['estacionamiento_desc'].isna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    #print(row[['url']])
    print(colorize_row(row)[['estacionamientos', 'estacionamiento_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    #print("2. Sustituir el valor de 'precio' por el de 'precio_desc'")
    print("3. Sustituir el valor manualmente")
    
    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'estacionamientos': ")
        df.at[idx, 'estacionamientos'] = int(new_value)
    #elif option == '2':
        # Sustituir el valor de 'estacionamientos' por 'estacionamiento_desc'
        #df.at[idx, 'estacionamientos'] = df.at[idx, 'estacionamiento_desc']
    elif option == '1':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")

    print("\n---\n")

print("Proceso completado.")

Proceso completado.


### Estacionamientos [0 / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['estacionamientos']==0) & (df['estacionamiento_desc'].notna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    #print(row[['estacionamientos', 'estacionamiento_desc', 'url']])
    print(colorize_row(row)[['estacionamientos', 'estacionamiento_desc', 'url']])

    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    print("2. Sustituir el valor de 'estacionamientos' por el de 'estacionamiento_desc'")
    print("3. Sustituir el valor manualmente")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'estacionamientos': ")
        df.at[idx, 'estacionamientos'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'estacionamientos'] = df.at[idx, 'estacionamiento_desc']
    elif option == '1':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")

Proceso completado.


### Estacionamientos [# / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['estacionamientos']!=0) & (df['estacionamiento_desc'].notna())) & (df['estacionamientos'] != df['estacionamiento_desc']) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    #print(row[['estacionamientos', 'estacionamiento_desc', 'url']])
    print(colorize_row(row)[['estacionamientos', 'estacionamiento_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    print("2. Sustituir el valor de 'estacionamientos' por el de 'estacionamiento_desc'")
    print("3. Sustituir el valor manualmente")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'estacionamientos': ")
        df.at[idx, 'estacionamientos'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'estacionamientos'] = df.at[idx, 'estacionamiento_desc']
    elif option == '1':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")

In [ ]:
shapes = df.shape
print('La base ha quedado con: ',f'{RED}{shapes}{RESET}', 'de',f'{GREEN}{inicio}{RESET}')

## b) Mantener rango de estacionamientos < 5 (mínimo 1)

In [ ]:
df = df[df['estacionamientos'] <= 8]
#df[['estacionamientos','estacionamiento_desc','url']]

# 8. Recámaras

## a) Recámaras en descripción

In [124]:
# Función para extraer cantidad de recamaras en descripción
def extraer_recamaras(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones de totales
    totales = re.findall(r'\b(\d+[\.,]?\d*)\s*(recamara|recamaras|habitaciones|dormitorio|dormitorios)\b', descripcion)
    total = None
    for match in totales:
        numero, _ = match
        numero = numero.replace(',', '').replace("'", '')
        try:
            total = float(numero)
        except ValueError:
            continue
        if total:
            break
    return total

# Aplicar la función al DataFrame
df['recamaras_desc'] = df['descripcion'].apply(extraer_recamaras)
inicio = df.shape

In [ ]:
# Aplicar colores a los valores de precio y precio_desc
def colorize_row(row):
    row_colored = row.copy()  # Crear una copia para no modificar el original
    row_colored['recamaras'] = f"{MAGENTA}{row['recamaras']}{RESET}"
    row_colored['recamaras_desc'] = f"{GREEN}{row['recamaras_desc']}{RESET}"
    return row_colored

### Recámaras [0 / 0]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[ (df['recamaras'] == 0 ) & (df['recamaras_desc'].isna()) ] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    #print(row[['url']])
    print(colorize_row(row)[['recamaras', 'recamaras_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    print("3. Sustituir el valor manualmente")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'recamaras': ")
        df.at[idx, 'recamaras'] = float(new_value)
    elif option == '1':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")

Proceso completado.


### Recámaras [0 / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['recamaras']==0) & (df['recamaras_desc'].notna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(colorize_row(row)[['recamaras', 'recamaras_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    print("2. Sustituir el valor de 'recamaras' por el de 'recamaras_desc'")
    print("3. Sustituir el valor manualmente")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'recamaras': ")
        df.at[idx, 'recamaras'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'recamaras'] = df.at[idx, 'recamaras_desc']
    elif option == '1':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Proceso completado.


### Recámaras [# / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['recamaras']!=0) & (df['recamaras_desc'].notna()) ) & (df['recamaras'] != df['recamaras_desc'])] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(colorize_row(row)[['recamaras', 'recamaras_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    print("2. Sustituir el valor de 'recamaras' por el de 'recamaras_desc'")
    print("3. Sustituir el valor manualmente")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'recamaras': ")
        df.at[idx, 'recamaras'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'recamaras'] = df.at[idx, 'recamaras_desc']
    elif option == '1':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


In [ ]:
shapes = df.shape
print('La base ha quedado con: ',f'{RED}{shapes}{RESET}', 'de',f'{GREEN}{inicio}{RESET}')

## b) Mantener rango de recámaras <= 7 

In [ ]:
print(df['recamaras'].unique())
df = df[(df['recamaras'] <= 7)] 
#print(df[['recamaras','recamaras_desc','url']])

# 9. Baños

## a) Baños en descripción

Manejando tanto los completos como los medios

In [125]:
# Extraer la cantidad de baños y medios baños en la descripción
def extraer_baños(descripcion):
    descripcion = descripcion.lower()
    
    # Buscar patrones de baños completos
    completos = re.findall(r'\b(\d+[\.,]?\d*)\s*baños?\b', descripcion)
    total_completos = sum([float(match.replace(',', '').replace("'", '')) for match in completos])
    
    # Buscar patrones de medios baños (1/2 baño)
    medios = re.findall(r'\b(\d+[\.,]?\d*)\s*medios?\s*baños?\b', descripcion)
    total_medios = sum([float(match.replace(',', '').replace("'", '')) for match in medios])
    
    # Retornar como un diccionario o una tupla
    return {'completos': total_completos, 'medios': total_medios}

# Aplicar la función al DataFrame
df['baños_desc'] = df['descripcion'].apply(extraer_baños)

# Si deseas separar en dos columnas distintas:
df['baños_completos'] = df['baños_desc'].apply(lambda x: x['completos'] if x else 0)
df['medios_baños'] = df['baños_desc'].apply(lambda x: x['medios'] if x else 0)

### Baños

In [ ]:
# Aplicar colores a los valores de precio y precio_desc
def colorize_row(row):
    row_colored = row.copy()  # Crear una copia para no modificar el original
    row_colored['banos'] = f"{MAGENTA}{row['banos']}{RESET}"
    row_colored['baños_completos'] = f"{GREEN}{row['baños_completos']}{RESET}"
    return row_colored
inicio = df.shape

#### [0 / 0]

Poner como mínimo para el registro 1 en baños, pero si cuenta con medio baño en descripción que se le asigne ese valor.

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[(df['banos'] == 0) & (df['baños_completos'].isna())]
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(colorize_row(row)[['banos', 'baños_completos', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    #print("2. Asignar 1 como mínimo y pasar al siguiente registro")
    print("3. Sustituir el valor manualmente")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'banos': ")
        df.at[idx, 'banos'] = float(new_value)
    elif option == '1':
        continue  # Pasar al siguiente registro
    #elif option == '2':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'banos'] = 1
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

Proceso completado.


#### [0 / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['banos']==0) & (df['baños_completos'].notna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(colorize_row(row)[['banos', 'baños_completos', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    print("2. Sustituir el valor de 'banos' por el de 'baños_completos'")
    print("3. Sustituir el valor manualmente")
    #print("4. Asignar 1 como mínimo y pasar al siguiente registro")
    

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2 | 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'banos': ")
        df.at[idx, 'banos'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'banos' por 'baños_completos'
        df.at[idx, 'banos'] = df.at[idx, 'baños_completos']
    elif option=='1':
        continue
    #elif option == '4':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'banos'] = 1
        #continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

#### [# / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['banos']!=0) & (df['baños_completos'].notna())) & (df['banos'] != df['baños_completos']))] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(colorize_row(row)[['banos', 'baños_completos', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    print("2. Sustituir el valor de 'banos' por el de 'baños_completos'")
    print("3. Sustituir el valor manualmente")
    #print("4. Asignar 1 como mínimo y pasar al siguiente registro")
    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2 | 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'banos': ")
        df.at[idx, 'banos'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'banos' por 'baños_completos'
        df.at[idx, 'banos'] = df.at[idx, 'baños_completos']
    elif option == '1':
        continue
    #elif option == '4':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'banos'] = 1
        #continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

In [ ]:
shapes = df.shape
print('La base ha quedado con: ',f'{RED}{shapes}{RESET}', 'de',f'{GREEN}{inicio}{RESET}')

### Medio baños

In [ ]:
# Aplicar colores a los valores de precio y precio_desc
def colorize_row(row):
    row_colored = row.copy()  # Crear una copia para no modificar el original
    row_colored['medio_banos'] = f"{MAGENTA}{row['medio_banos']}{RESET}"
    row_colored['medios_baños'] = f"{GREEN}{row['medios_baños']}{RESET}"
    return row_colored
inicio = df.shape

#### [0 / 0]

Poner como mínimo para el registro 1 en baños, pero si cuenta con valor en descripción que se le asigne ese valor.

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[(df['medio_banos'] == 0) & (df['medios_baños'].isna())]
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(colorize_row(row)[['medio_banos', 'medios_baños', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    #print("2. Asignar 1 como mínimo y pasar al siguiente registro")
    print("3. Sustituir el valor manualmente")
    
    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'medio_banos': ")
        df.at[idx, 'medio_banos'] = float(new_value)
    #elif option == '2':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'banos'] = 1
    elif option=='1':
        continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

#### [0 / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['medio_banos']==0) & (df['medios_baños'].notna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(colorize_row(row)[['medio_banos', 'medios_baños', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    print("2. Sustituir el valor de 'medio_banos' por el de 'medios_baños'")
    #print("3. Asignar 1 como mínimo y pasar al siguiente registro")
    print("3. Sustituir el valor manualmente")
    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2 | 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'medio_banos': ")
        df.at[idx, 'medio_banos'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'medio_banos' por 'medios_baños'
        df.at[idx, 'medio_banos'] = df.at[idx, 'medios_baños']
    #elif option == '3':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'medio_baños'] = 1
    elif option=='1':
        continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

#### [# / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['medio_banos']!=0) & (df['medios_baños'].notna())) & (df['medio_banos'] != df['medios_baños']))] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(colorize_row(row)[['medio_banos', 'medios_baños', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    print("2. Sustituir el valor de 'medio_banos' por el de 'medios_baños'")
    #print("4. Asignar 1 como mínimo y pasar al siguiente registro")
    print("3. Sustituir el valor manualmente")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2 | 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'medio_banos': ")
        df.at[idx, 'medio_banos'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'medio_baños' por 'medios_baños'
        df.at[idx, 'medio_banos'] = df.at[idx, 'medios_baños']
    #elif option == '4':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'medio_baños'] = 1
    elif option=='1':
        continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

In [ ]:
shapes = df.shape
print('La base ha quedado con: ',f'{RED}{shapes}{RESET}', 'de',f'{GREEN}{inicio}{RESET}')

## B) Total de baños

In [126]:
# Make the medio_baños and banos sum
# 'medio_baños', divididos entre 2 para representar medios
df['bano_total'] = df['banos'] + (df['medio_banos'] / 2)
df['bano_total'] = df['bano_total'].round(1)  # Round the total, not just 'banos'
print(df[['banos', 'medio_banos', 'bano_total']].head())

   banos  medio_banos  bano_total
0      1            0         1.0
1      1            0         1.0
2      1            0         1.0
3      2            0         2.0
4      2            0         2.0


# 10. Ubicación

## a) Desde descripción

In [130]:
# Función para extraer ubicación de la descripción
def extraer_ubicacion(descripcion):
    descripcion = descripcion.lower()
    # Patrones para buscar ubicaciones comunes
    patrones = [
        r'(?:ubicad[oa] en|en) ([\w\s]+(?:,\s*[\w\s]+){0,2})',
        r'(?:calle|avenida|av\.) ([\w\s]+(?:,\s*[\w\s]+){0,2})',
        r'(?:colonia|col\.) ([\w\s]+(?:,\s*[\w\s]+){0,2})',
        r'(?:fraccionamiento|fracc\.) ([\w\s]+(?:,\s*[\w\s]+){0,2})'
    ]
    2.
    for patron in patrones:
        match = re.search(patron, descripcion)
        if match:
            return match.group(1).strip().title()
    
    return None

# Aplicar la función al DataFrame
df['ubicacion_desc'] = df['descripcion'].apply(extraer_ubicacion)

#### Cuando la ubicación dice centro:

In [131]:
# Filtrar los registros donde la ubicación contiene la palabra "centro"
df_centro = df[df['ubicacion'].str.contains('centro|Centro', case=False, na=False)]
num_registros_centro = len(df_centro)

print(f"Número de registros que contienen 'centro' en la ubicación:",f"{RED} {num_registros_centro}{RESET}")

Número de registros que contienen 'centro' en la ubicación:  142


In [ ]:
# Aplicar colores a los valores de precio y precio_desc
def colorize_row(row):
    row_colored = row.copy()  # Crear una copia para no modificar el original
    row_colored['ubicacion'] = f"{MAGENTA}{row['ubicacion']}{RESET}"
    row_colored['ubicacion_desc'] = f"{GREEN}{row['ubicacion_desc']}{RESET}"
    return row_colored
inicio = df.shape

In [ ]:

pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in df_centro.iterrows():
    print(f"Registro {idx}:")    
    #print(row[['ubicacion', 'ubicacion_desc', 'url']])
    print(colorize_row(row)[['ubicacion', 'ubicacion_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Pasar al siguiente registro")
    print("2. Sustituir el valor de 'ubicacion' por el de 'ubicacion_desc'")
    print("3. Sustituir el valor manualmente")
    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '3':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'ubicacion': ")
        df.at[idx, 'ubicacion'] = new_value  
    elif option == '2':
        # Sustituir el valor de 'ubicacion' por 'ubicacion_desc'
        df.at[idx, 'ubicacion'] = df.at[idx, 'ubicacion_desc']
    elif option == '1':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")

## b) Separación

Se divide la ubicación por 
* Colonia
* Municipio
* Estado

In [5]:
# Siempre regresar una lista de 3 elementos
def split_location(location):
    parts = location.split(',')
    parts = [part.strip() for part in parts]  # Eliminar espacios al principio y final de cada parte
    parts = [unicodedata.normalize('NFKD', part).encode('ascii', 'ignore').decode('utf-8') for part in parts]  # Eliminar acentos y otros caracteres no ascii
    parts += [None] * (3 - len(parts))  # Asegurar que siempre hay 3 partes
    return parts[:3]  # Devolver solo las primeras 3 partes

df[['Colonia', 'Municipio', 'Estado']] = df['ubicacion'].apply(split_location).apply(pd.Series)

# Limpiando valores nulos
df['Colonia'] = df['Colonia'].str.strip().fillna('Sin_colonia')
df['Municipio'] = df['Municipio'].str.strip().fillna('Sin_municipio')
df['Estado'] = df['Estado'].str.strip().fillna('Sin_estado')
df[['ubicacion', 'Colonia', 'Municipio', 'Estado']]

,ubicacion,Colonia,Municipio,Estado
0,"Puebla Centro, Puebla, Puebla",Puebla Centro,Puebla,Puebla
1,"Las Hadas Mundial 86, Puebla, Puebla",Las Hadas Mundial 86,Puebla,Puebla
2,"Plazas de Amalucan, Puebla, Puebla",Plazas de Amalucan,Puebla,Puebla
3,"Héroes de Puebla, Puebla, Puebla",Heroes de Puebla,Puebla,Puebla
4,"Plazas de Amalucan, Puebla, Puebla",Plazas de Amalucan,Puebla,Puebla
...,...,...,...,...
3471,"Puebla, Puebla, Puebla",Puebla,Puebla,Puebla
3472,"Angelópolis, Puebla, Puebla",Angelopolis,Puebla,Puebla
3473,"La Paz, Puebla, Puebla",La Paz,Puebla,Puebla
3474,"San José Mayorazgo, Puebla, Puebla",San Jose Mayorazgo,Puebla,Puebla


In [6]:
incompletos = df[(df['Municipio'].isna() | df['Estado'].isna()) & ~(df['Municipio'].isna() & df['Estado'].isna())]
incompletos

,id,categoria,precio,precio_mxn,precio_usd,fecha_conversion,propiedad,metros_total,metros_construido,precio_m2_terreno,...,bano_total,seguridad_privada,tipo_lugar,Colonia,Municipio,Estado,CP,ubicacion,url,descripcion


In [7]:
# Aplicando valor de acuerdo a la base a limpiar
#df['Estado'] = df['Estado'].replace('Sin_estado', 'Hidalgo')

Estandarizar valores

In [8]:
# Minúsculas
df['Colonia'] = df['Colonia'].str.strip().str.lower()
df['Municipio'] = df['Municipio'].str.strip().str.lower()
df['Estado'] = df['Estado'].str.strip().str.lower()

# Esta línea de código aplica la función unicodedata.normalize a la columna 'Colonia' del DataFrame df.
# El argumento 'NFKD' indica que se debe aplicar la normalización NFKD, que elimina los acentos y otros caracteres diacríticos.
#df['Colonia'] = df['Colonia'].apply(lambda x: unicodedata.normalize('NFKD', x))

import unicodedata# Función para eliminar acentos
def eliminar_acentos(texto):
    # Normaliza y elimina los acentos
    texto_normalizado = unicodedata.normalize('NFKD', texto)
    return ''.join(char for char in texto_normalizado if unicodedata.category(char) != 'Mn')
df['Colonia'] = df['Colonia'].apply(eliminar_acentos)
df['Municipio'] = df['Municipio'].apply(eliminar_acentos)
df['Estado'] = df['Estado'].apply(eliminar_acentos)
print(df['Colonia'].unique())

['puebla centro' 'las hadas mundial 86' 'plazas de amalucan'
 'heroes de puebla' 'san jose mayorazgo' 'ex-hacienda mayorazgo'
 'la calera' 'el carmen' 'santa maria' 'puebla' 'santa barbara'
 'san isidro castillotla' 'la loma' 'santa maria la ribera'
 'villa las flores' 'el cerrito' 'la joya' 'san diego castillotla'
 'santa clara la venta' 'san miguel mayorazgo infonavit'
 'jardines de san manuel' 'loma linda' 'bosques del pilar' 'valle del sol'
 'bosques de chapultepec' 'club de golf puebla' 'granjas san isidro'
 'san angel' 'granjas del sur' 'villa frontera' 'banco de puebla'
 'bosques de san sebastian' 'cuauhtemoc' 'los cipreses' 'aquiles serdan'
 'unidad satelite magisterial' 'san bartolo infonavit' 'lomas del valle'
 'villa carmel' 'las brisas' 'las palmas' 'revolucion mexicana'
 'la hacienda' 'rivera de santiago' 'los heroes' 'geo villas las garzas'
 'san juan bosco' '3 cerritos' 'san ramon 4a secc' 'bosques amalucan'
 'la paz' 'plazas de guadalupe' 'vista hermosa' 'arboledas de g

#### Separación de tipo de lugar del nombre de la colonia

In [9]:
# Añade las diferentes formas de identificar el tipo de lugar (etiquetas de ubicación)
tipos = ['condominio', 'fraccionamiento', 'infonavit','colonia', 'frac','fracc','fraccio','fracc.','namiento','ionamiento', 'uhp', 'urbanización','urbanizacion', 'sector', 'barrio', 'zona', 'sector', 'unidad habitacional', ]
#, 'residencial','unidad residencial'
def obtener_tipo_lugar(nombre):
    for tipo in tipos:
        if tipo in nombre:
            return tipo
    return None

df['tipo_lugar'] = df['Colonia'].apply(obtener_tipo_lugar)
# Eliminar el tipo de lugar de la columna 'colonia'
df['Colonia'] = df.apply(lambda row: row['Colonia'].replace(row['tipo_lugar'], '').strip() if row['tipo_lugar'] else row['Colonia'], axis=1)
print(df[['Colonia','tipo_lugar','ubicacion']])
print(df['tipo_lugar'].unique())

                   Colonia tipo_lugar                             ubicacion
0            puebla centro       None         Puebla Centro, Puebla, Puebla
1     las hadas mundial 86       None  Las Hadas Mundial 86, Puebla, Puebla
2       plazas de amalucan       None    Plazas de Amalucan, Puebla, Puebla
3         heroes de puebla       None      Héroes de Puebla, Puebla, Puebla
4       plazas de amalucan       None    Plazas de Amalucan, Puebla, Puebla
...                    ...        ...                                   ...
3471                puebla       None                Puebla, Puebla, Puebla
3472           angelopolis       None           Angelópolis, Puebla, Puebla
3473                la paz       None                La Paz, Puebla, Puebla
3474    san jose mayorazgo       None    San José Mayorazgo, Puebla, Puebla
3475      centro historico       None      Centro Histórico, Puebla, Puebla

[3476 rows x 3 columns]
[None 'infonavit' 'barrio' 'zona']


In [10]:
df['id'] = range(1, len(df) + 1)

## c) Estandarizar valores 

### 1. Mapeo  

### Diccionario estandarizado

In [11]:
# Carga el diccionario de colonias estandarizadas desde un archivo CSV
diccionario = pd.read_csv('colonias_estandarizadas.csv')
print(diccionario.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8121 entries, 0 to 8120
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ubicacion              8121 non-null   object 
 1   Colonia                8121 non-null   object 
 2   Municipio              8121 non-null   object 
 3   Estado                 8121 non-null   object 
 4   tipo_lugar             666 non-null    object 
 5   Colonia_Estandarizada  8121 non-null   object 
 6   codigo_postal          8121 non-null   float64
dtypes: float64(1), object(6)
memory usage: 444.2+ KB
None


### Diccionario CP

In [12]:
archivo_excel = 'D:/yoe11/Documents/TESEO/Modelo Factible/Scraplining/CPdescarga.xlsx'
hojas_excel = pd.ExcelFile(archivo_excel, engine='openpyxl')
# Imprimir los nombres de las hojas
print(hojas_excel.sheet_names)

['Nota', 'Aguascalientes', 'Baja_California', 'Baja_California_Sur', 'Campeche', 'Coahuila_de_Zaragoza', 'Colima', 'Chiapas', 'Chihuahua', 'Distrito_Federal', 'Durango', 'Guanajuato', 'Guerrero', 'Hidalgo', 'Jalisco', 'México', 'Michoacán_de_Ocampo', 'Morelos', 'Nayarit', 'Nuevo_León', 'Querétaro', 'Oaxaca', 'Puebla', 'Quintana_Roo', 'Sonora', 'San_Luis_Potosí', 'Sinaloa', 'Tabasco', 'Tamaulipas', 'Tlaxcala', 'Veracruz_de_Ignacio_de_la_Llave', 'Yucatán', 'Zacatecas']


In [13]:
# Obtener el DataFrame específico para el estado 'Puebla'
df_codigos_postales = pd.read_excel(archivo_excel, sheet_name='Puebla', engine='openpyxl', usecols=['D_mnpio', 'd_asenta', 'd_tipo_asenta', 'd_codigo'])
df_codigos_postales['estado'] = 'Puebla'  # Agregar el nombre del estado como 'estado'
df_codigos_postales.head()

,d_codigo,d_asenta,d_tipo_asenta,D_mnpio,estado
0,72000,Centro,Colonia,Puebla,Puebla
1,72000,San Francisco,Colonia,Puebla,Puebla
2,72010,Santa María la Rivera,Colonia,Puebla,Puebla
3,72010,INFONAVIT Bosques 5 de Febrero,Unidad habitacional,Puebla,Puebla
4,72010,Residencial Santa María,Unidad habitacional,Puebla,Puebla


In [14]:
#df_codigos_postales = df_codigos_postales[df_codigos_postales['D_mnpio'].str.contains('Pachuca de Soto')]
#df_codigos_postales

In [15]:
# Limpiar y estandarizar las columnas
df_codigos_postales['D_mnpio'] = df_codigos_postales['D_mnpio'].str.lower().str.strip()
df_codigos_postales['d_asenta'] = df_codigos_postales['d_asenta'].str.lower().str.strip()
df_codigos_postales['d_tipo_asenta'] = df_codigos_postales['d_tipo_asenta'].str.lower().str.strip()
df_codigos_postales['estado'] = df_codigos_postales['estado'].str.lower().str.strip()
# Eliminar acentos de todas las columnas de df_codigos_postales para que se pueda mapear bien
df_codigos_postales['d_asenta'] = df_codigos_postales['d_asenta'].apply(lambda x: unicodedata.normalize('NFKD', x).encode('ascii', errors='ignore').decode('utf-8') if isinstance(x, str) else x)

#### Comparación con diccionario

In [16]:
def estand_col(colonia, diccionario):
    # Busca en el diccionario si la colonia existe y devuelve su nombre estandarizado
    # Si no existe, devuelve el nombre original de la colonia
    return diccionario[diccionario['Colonia'] == colonia]['Colonia_Estandarizada'].values[0] if colonia in diccionario['Colonia'].values else colonia

# Aplica la función de estandarización a la columna 'Colonia' del DataFrame df
df['Colonia_Estandarizada'] = df['Colonia'].apply(lambda x: estand_col(x, diccionario))
df[['Colonia_Estandarizada','Colonia']]

,Colonia_Estandarizada,Colonia
0,centro,puebla centro
1,las hadas mundial 86,las hadas mundial 86
2,plazas amalucan,plazas de amalucan
3,heroes de puebla,heroes de puebla
4,plazas amalucan,plazas de amalucan
...,...,...
3471,puebla,puebla
3472,angelopolis,angelopolis
3473,la paz,la paz
3474,san jose mayorazgo,san jose mayorazgo


In [17]:
# Función para obtener el código postal dado estado, municipio, y colonia
def obtener_codigo_postal(colonia, municipio, estado, df_codigos_postales):
    # Normalizar los inputs
    estado = estado.lower().strip()
    municipio = municipio.lower().strip()
    colonia = colonia.lower().strip()
    
    estado = estado.lower().strip().translate(str.maketrans('', '', 'áéíóúÁÉÍÓÚ'))
    municipio = municipio.lower().strip().translate(str.maketrans('', '', 'áéíóúÁÉÍÓÚ'))
    colonia = colonia.lower().strip().translate(str.maketrans('', '', 'áéíóúÁÉÍÓÚ'))

    # Filtrar el DataFrame según estado y municipio
    df_filtrado = df_codigos_postales[(df_codigos_postales['estado'] == estado) & (df_codigos_postales['D_mnpio'] == municipio)]
    
    # Intentar encontrar coincidencias exactas de la colonia
    resultado = df_filtrado[df_filtrado['d_asenta'] == colonia]
    
    if not resultado.empty:
        # Si se encuentra una coincidencia, devolver el código postal
        return resultado['d_codigo'].values[0]
    
    return None  # Devolver None si no hay coincidencia

In [18]:
df['codigo_postal'] = df.apply(lambda row: obtener_codigo_postal(row['Colonia_Estandarizada'], row['Municipio'], row['Estado'], df_codigos_postales), axis=1)

# Verificar si hay registros sin código postal asignado
sin_codigo_p = df[df['codigo_postal'].isna()]
Limpio = df[df['codigo_postal'].notna()]
print(sin_codigo_p[['Colonia','Municipio','Estado','codigo_postal']])
print(sin_codigo_p.shape)
print(Limpio.shape)

                       Colonia Municipio      Estado  codigo_postal
242   hacienda del camino real    puebla      puebla            NaN
306                       snte    puebla      puebla            NaN
356          villas de atlixco    puebla      puebla            NaN
357          villas de atlixco    puebla      puebla            NaN
387                     puebla    puebla  sin_estado            NaN
...                        ...       ...         ...            ...
3417                    puebla    puebla  sin_estado            NaN
3418             la concepcion    puebla      puebla            NaN
3446      residencial el pilar    puebla      puebla            NaN
3453    geo villas atlixcayotl    puebla      puebla            NaN
3457    geo villas atlixcayotl    puebla      puebla            NaN

[228 rows x 4 columns]
(228, 32)
(3248, 32)


### Primera: 
* Por mapeo en coincidencia flexible (fuzzy)

In [19]:
# Función para mapear colonias con coincidencia flexible
def mapear_colonia_flexible(colonia, df_codigos_postales):
    # Lista de todas las colonias en el diccionario
    colonias_diccionario = df_codigos_postales['d_asenta'].unique()

    # Buscar la coincidencia más cercana con fuzzywuzzy
    mejor_coincidencia = process.extractOne(colonia, colonias_diccionario, scorer=fuzz.ratio, score_cutoff=80)  # Ajusta el threshold de similitud a 80
    
    # Si se encuentra una coincidencia, devolverla; de lo contrario, devolver el valor original
    return mejor_coincidencia[0] if mejor_coincidencia else colonia
# Crear diccionario
# Aplicar la función al DataFrame original
#df['Colonia_Estandarizada'] = df['Colonia'].apply(lambda x: mapear_colonia_flexible(x, df_codigos_postales))
#df[['Colonia_Estandarizada','Colonia']]

# Con diccionario disponible
# Aplicar la función al DataFrame original
sin_codigo_p['Colonia_Estandarizada'] = sin_codigo_p['Colonia'].apply(lambda x: mapear_colonia_flexible(x, df_codigos_postales))
print(sin_codigo_p[['tipo_lugar', 'Colonia', 'Colonia_Estandarizada', 'Municipio', 'Estado', 'ubicacion']])

     tipo_lugar                   Colonia     Colonia_Estandarizada Municipio  \
242        None  hacienda del camino real  hacienda del camino real    puebla   
306        None                      snte                      snte    puebla   
356        None         villas de atlixco         villas de atlixco    puebla   
357        None         villas de atlixco         villas de atlixco    puebla   
387        None                    puebla                    puebla    puebla   
...         ...                       ...                       ...       ...   
3417       None                    puebla                    puebla    puebla   
3418       None             la concepcion             la concepcion    puebla   
3446       None      residencial el pilar      residencial el pilar    puebla   
3453       None    geo villas atlixcayotl     geovillas atlixcayotl    puebla   
3457       None    geo villas atlixcayotl     geovillas atlixcayotl    puebla   

          Estado           

C:\Users\yoe11\AppData\Local\Temp\ipykernel_17044\2994268618.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sin_codigo_p['Colonia_Estandarizada'] = sin_codigo_p['Colonia'].apply(lambda x: mapear_colonia_flexible(x, df_codigos_postales))


In [20]:
sin_codigo_p['codigo_postal'] = sin_codigo_p.apply(lambda row: obtener_codigo_postal(row['Colonia_Estandarizada'], row['Municipio'], row['Estado'], df_codigos_postales), axis=1)
# Verificar si hay registros sin código postal asignado

sin_codigo_postal = sin_codigo_p[sin_codigo_p['codigo_postal'].isna()]
Limpios = sin_codigo_p[sin_codigo_p['codigo_postal'].notna()]
print(sin_codigo_postal[['Colonia','Municipio','Estado','codigo_postal']])
print(sin_codigo_postal.shape)

                       Colonia Municipio      Estado codigo_postal
242   hacienda del camino real    puebla      puebla          None
306                       snte    puebla      puebla          None
356          villas de atlixco    puebla      puebla          None
357          villas de atlixco    puebla      puebla          None
387                     puebla    puebla  sin_estado          None
...                        ...       ...         ...           ...
3417                    puebla    puebla  sin_estado          None
3418             la concepcion    puebla      puebla          None
3446      residencial el pilar    puebla      puebla          None
3453    geo villas atlixcayotl    puebla      puebla          None
3457    geo villas atlixcayotl    puebla      puebla          None

[228 rows x 4 columns]
(228, 32)


C:\Users\yoe11\AppData\Local\Temp\ipykernel_17044\2139016896.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sin_codigo_p['codigo_postal'] = sin_codigo_p.apply(lambda row: obtener_codigo_postal(row['Colonia_Estandarizada'], row['Municipio'], row['Estado'], df_codigos_postales), axis=1)


SEGUNDA ESTANDA
coincidencias parciales

In [21]:
# Mapear colonias de df_codigos_postales a las colonias de df para estandarizar

# Coincidencias exactas
#sin_codigo_postal['Colonia_Estandarizada'] = sin_codigo_postal['Colonia'].apply(lambda x: df_codigos_postales[df_codigos_postales['d_asenta'].str.lower() == x.lower()]['d_asenta'].values[0] if not df_codigos_postales[df_codigos_postales['d_asenta'].str.lower() == x.lower()].empty else x)
# Coincidencias parciales
sin_codigo_postal.loc[:, 'Colonia_Estandarizada'] = sin_codigo_postal['Colonia'].apply(lambda x: df_codigos_postales[df_codigos_postales['d_asenta'].str.contains(x, case=False)]['d_asenta'].values[0] if not df_codigos_postales[df_codigos_postales['d_asenta'].str.contains(x, case=False)].empty else x)
print(sin_codigo_postal[['Colonia','Colonia_Estandarizada','Municipio','Estado','ubicacion']].head(40))

                       Colonia          Colonia_Estandarizada  \
242   hacienda del camino real       hacienda del camino real   
306                       snte                           snte   
356          villas de atlixco              villas de atlixco   
357          villas de atlixco              villas de atlixco   
387                     puebla              angeles de puebla   
446   hacienda del camino real       hacienda del camino real   
447   hacienda del camino real       hacienda del camino real   
477               las 3 cruces                   las 3 cruces   
511          hacienda zavaleta              hacienda zavaleta   
522             villa olimpica                 villa olimpica   
648         san vicente ferrer             san vicente ferrer   
654       lomas de angelopolis  lomas de angelopolis privanza   
670                     puebla              angeles de puebla   
821     fuentes de angelopolis         fuentes de angelopolis   
822                     p

In [22]:
sin_codigo_postal.loc[:, 'codigo_postal'] = sin_codigo_postal.apply(lambda row: obtener_codigo_postal(row['Colonia_Estandarizada'], row['Municipio'], row['Estado'], df_codigos_postales), axis=1)
# Verificar si hay registros sin código postal asignado
sin_codigo_postal_2 = sin_codigo_postal[sin_codigo_postal['codigo_postal'].isna()]
Limpios2 = sin_codigo_postal[sin_codigo_postal['codigo_postal'].notna()]

print(sin_codigo_postal_2[['Colonia','Municipio','Estado','codigo_postal']])
print(sin_codigo_postal_2.shape)
print(Limpios2.shape)

                       Colonia Municipio      Estado codigo_postal
242   hacienda del camino real    puebla      puebla          None
306                       snte    puebla      puebla          None
356          villas de atlixco    puebla      puebla          None
357          villas de atlixco    puebla      puebla          None
387                     puebla    puebla  sin_estado          None
...                        ...       ...         ...           ...
3417                    puebla    puebla  sin_estado          None
3418             la concepcion    puebla      puebla          None
3446      residencial el pilar    puebla      puebla          None
3453    geo villas atlixcayotl    puebla      puebla          None
3457    geo villas atlixcayotl    puebla      puebla          None

[228 rows x 4 columns]
(228, 32)
(0, 32)


### Convirtiendo números a palabras y coincidencias parciales segunda vez

In [23]:
# Diccionario para convertir números cardinales a palabras
numeros_cardinales = {
    '1': 'uno',
    '2': 'dos',
    '3': 'tres',
    '4': 'cuatro',
    '5': 'cinco',
    '6': 'seis',
    '7': 'siete',
    '8': 'ocho',
    '9': 'nueve',
    '10': 'diez',
}

# Diccionario para convertir números ordinales a palabras
numeros_ordinales = {
    '1ra': 'primera',
    '2da': 'segunda',
    '3ra': 'tercera',
    '4ta': 'cuarta',
    '5ta': 'quinta',
    '6ta': 'sexta',
    '7ma': 'séptima',
    '8va': 'octava',
    '9na': 'novena',
    '10ma': 'décima',
    # Agregar más ordinales si es necesario
}
import re

def reemplazar_numeros(colonia):
    # Primero reemplazar los ordinales
    for num_ord, palabra_ord in numeros_ordinales.items():
        colonia = re.sub(rf'\b{num_ord}\b', palabra_ord, colonia)
    
    # Luego reemplazar los números cardinales aislados
    for num_card, palabra_card in numeros_cardinales.items():
        colonia = re.sub(rf'\b{num_card}\b', palabra_card, colonia)
    
    return colonia

# Utilizar .loc para asignar el resultado de la aplicación de la función reemplazar_numeros
sin_codigo_postal_2.loc[:, 'Colonia_Estandarizada'] = sin_codigo_postal_2['Colonia'].apply(reemplazar_numeros)
sin_codigo_postal_2['Colonia_Estandarizada'].unique()

array(['hacienda del camino real', 'snte', 'villas de atlixco', 'puebla',
       'las tres cruces', 'hacienda zavaleta', 'villa olimpica',
       'san vicente ferrer', 'lomas de angelopolis',
       'fuentes de angelopolis', 'residencial puebla',
       'agricola villa albertina', 'las margaritas',
       'residencial zavaleta', 'rincon de san andres',
       'bugambilias 3a secc', 'el barreal', 'geo villas atlixcayotl',
       'momosa', 'pueblo nuevo', 'gral. ignacio zaragoza',
       'villas san carlos', 'camino real a cholula', '20 de noviembre',
       'momoxpan ii', 'concepcion la cruz', 'concepcion guadalupe',
       'residencial del encanto', 'francisco i madero', 'villa la noria',
       'villas zavaleta primera seccion', 'santa gema',
       'villa de zavaleta', 'altavista', 'la concepcion',
       'las fuentes de puebla', 'insurgentes zaragoza',
       'residencial el pilar'], dtype=object)

In [24]:
sin_codigo_postal_2.loc[:, 'codigo_postal'] = sin_codigo_postal_2.apply(lambda row: obtener_codigo_postal(row['Colonia_Estandarizada'], row['Municipio'], row['Estado'], df_codigos_postales), axis=1)
# Verificar si hay registros sin código postal asignado
sin_codigo_postal_3 = sin_codigo_postal_2[sin_codigo_postal_2['codigo_postal'].isna()]
limpios3 = sin_codigo_postal_2[sin_codigo_postal_2['codigo_postal'].notna()]

print(sin_codigo_postal_3[['Colonia','Municipio','Estado','codigo_postal']])
print(sin_codigo_postal_3.shape)
print(limpios3.shape)

                       Colonia Municipio      Estado codigo_postal
242   hacienda del camino real    puebla      puebla          None
306                       snte    puebla      puebla          None
356          villas de atlixco    puebla      puebla          None
357          villas de atlixco    puebla      puebla          None
387                     puebla    puebla  sin_estado          None
...                        ...       ...         ...           ...
3417                    puebla    puebla  sin_estado          None
3418             la concepcion    puebla      puebla          None
3446      residencial el pilar    puebla      puebla          None
3453    geo villas atlixcayotl    puebla      puebla          None
3457    geo villas atlixcayotl    puebla      puebla          None

[228 rows x 4 columns]
(228, 32)
(0, 32)


### Limpieza final manual para actualizar el diccionario 

In [25]:
df_correctos = pd.concat([Limpio, Limpios, Limpios2, limpios3])
print(df_correctos.shape)
print(df_correctos.info())

(3248, 32)
<class 'pandas.core.frame.DataFrame'>
Index: 3248 entries, 0 to 3475
Data columns (total 32 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         3248 non-null   int64  
 1   categoria                  3029 non-null   object 
 2   precio                     3248 non-null   float64
 3   precio_mxn                 3248 non-null   float64
 4   precio_usd                 3248 non-null   int64  
 5   fecha_conversion           3248 non-null   object 
 6   propiedad                  3248 non-null   object 
 7   metros_total               3248 non-null   int64  
 8   metros_construido          3248 non-null   int64  
 9   precio_m2_terreno          3248 non-null   float64
 10  precio_m2_construido       3248 non-null   float64
 11  tiempo_de_publicacion      3248 non-null   object 
 12  meses_transcurridos        3248 non-null   int64  
 13  meses_transcurridos_fecha  3248 non-null  

C:\Users\yoe11\AppData\Local\Temp\ipykernel_17044\1932133056.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_correctos = pd.concat([Limpio, Limpios, Limpios2, limpios3])


In [26]:
registros_sin_codigo_postal = df[df['codigo_postal'].isna()]
shape_df = df.shape[0]
correct = df_correctos.shape[0]
sin_code = registros_sin_codigo_postal.shape[0]

print("Total de registros en DF original",f"{RED}{shape_df}{RESET}")
print("Registros sin código postal en df:", f'{MAGENTA}{sin_code}{RESET}')

print("Total de registros en df_correctos:", f'{BLUE}{correct}{RESET}')
print("Registros con código postal en df_correctos:", df_correctos[df_correctos['codigo_postal'].notna()].shape[0])
restantes = shape_df - correct
print("Restantes: ",f'{GREEN}{restantes}{RESET}')

Total de registros en DF original 3476
Registros sin código postal en df: 228
Total de registros en df_correctos: 3248
Registros con código postal en df_correctos: 3248
Restantes:  228


In [27]:
# Crear un diccionario para almacenar las estandarizaciones previas
estandarizaciones_previas = {}
# Aplicar colores a los valores 
def colorize_row(row):
    row_colored = row.copy()  # Crear una copia para no modificar el original
    row_colored['ubicacion'] = f"{MAGENTA}{row['ubicacion']}{RESET}"
    row_colored['Colonia_Estandarizada'] = f"{BLUE}{row['Colonia_Estandarizada']}{RESET}"
    return row_colored
# Iterar sobre los registros sin código postal
for idx, row in registros_sin_codigo_postal.iterrows():
    print(f"Registro {idx}:")
    print(colorize_row(row)[['ubicacion','Estado','Municipio','Colonia','Colonia_Estandarizada']])
    # Verificar si ya existe una estandarización previa para la colonia actual
    colonia_actual = row['Colonia']
    if colonia_actual in estandarizaciones_previas:
        # Si ya existe, usar el valor previo
        new_value = estandarizaciones_previas[colonia_actual]
        print(f"Usando estandarización previa para '{colonia_actual}': {new_value}")
    else:
        # Solicitar una nueva estandarización si no existe en el diccionario
        new_value = input("Introduce estandarización de Colonia: ").lower()
        estandarizaciones_previas[colonia_actual] = new_value  # Guardar la nueva estandarización
    # Asignar el valor estandarizado al DataFrame
    registros_sin_codigo_postal.at[idx, 'Colonia_Estandarizada'] = new_value
    print("Cambio realizado por: ", new_value)
    print("\n---\n")
print("Proceso completado.")

Registro 242:
ubicacion                Hacienda Del Camino Real, Puebla, Puebla
Estado                                                              puebla
Municipio                                                           puebla
Colonia                                           hacienda del camino real
Colonia_Estandarizada                    hacienda del camino real
Name: 242, dtype: object
Cambio realizado por:  camino real

---

Registro 306:
ubicacion                SNTE, Puebla, Puebla
Estado                                          puebla
Municipio                                       puebla
Colonia                                           snte
Colonia_Estandarizada                    snte
Name: 306, dtype: object
Cambio realizado por:  snte

---

Registro 356:
ubicacion                Villas de Atlixco, Puebla, Puebla
Estado                                                       puebla
Municipio                                                    puebla
Colonia                 

In [28]:
registros_sin_codigo_postal['codigo_postal'] = registros_sin_codigo_postal.apply(lambda row: obtener_codigo_postal(row['Colonia_Estandarizada'], row['Municipio'], row['Estado'], df_codigos_postales), axis=1)

# Verificar si hay registros sin código postal asignado
sin_codigo_f = registros_sin_codigo_postal[registros_sin_codigo_postal['codigo_postal'].isna()]
Limpio_fin = registros_sin_codigo_postal[registros_sin_codigo_postal['codigo_postal'].notna()]
print(sin_codigo_f[['Colonia','Municipio','Estado','codigo_postal']])
print(sin_codigo_f.shape)
print(Limpio_fin.shape)

C:\Users\yoe11\AppData\Local\Temp\ipykernel_17044\4012415720.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  registros_sin_codigo_postal['codigo_postal'] = registros_sin_codigo_postal.apply(lambda row: obtener_codigo_postal(row['Colonia_Estandarizada'], row['Municipio'], row['Estado'], df_codigos_postales), axis=1)


                     Colonia Municipio      Estado  codigo_postal
306                     snte    puebla      puebla            NaN
356        villas de atlixco    puebla      puebla            NaN
357        villas de atlixco    puebla      puebla            NaN
387                   puebla    puebla  sin_estado            NaN
511        hacienda zavaleta    puebla      puebla            NaN
...                      ...       ...         ...            ...
3417                  puebla    puebla  sin_estado            NaN
3418           la concepcion    puebla      puebla            NaN
3446    residencial el pilar    puebla      puebla            NaN
3453  geo villas atlixcayotl    puebla      puebla            NaN
3457  geo villas atlixcayotl    puebla      puebla            NaN

[198 rows x 4 columns]
(198, 32)
(30, 32)


CREAR DICCIONARIO

In [29]:
df_correctos = pd.concat([Limpio, Limpios, Limpios2, limpios3,Limpio_fin])
print(df_correctos.shape)
print(df_correctos.info())

C:\Users\yoe11\AppData\Local\Temp\ipykernel_17044\3680138287.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_correctos = pd.concat([Limpio, Limpios, Limpios2, limpios3,Limpio_fin])


(3278, 32)
<class 'pandas.core.frame.DataFrame'>
Index: 3278 entries, 0 to 3157
Data columns (total 32 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         3278 non-null   int64  
 1   categoria                  3057 non-null   object 
 2   precio                     3278 non-null   float64
 3   precio_mxn                 3278 non-null   float64
 4   precio_usd                 3278 non-null   int64  
 5   fecha_conversion           3278 non-null   object 
 6   propiedad                  3278 non-null   object 
 7   metros_total               3278 non-null   int64  
 8   metros_construido          3278 non-null   int64  
 9   precio_m2_terreno          3278 non-null   float64
 10  precio_m2_construido       3278 non-null   float64
 11  tiempo_de_publicacion      3278 non-null   object 
 12  meses_transcurridos        3278 non-null   int64  
 13  meses_transcurridos_fecha  3278 non-null  

In [30]:
df_correctos.columns

Index(['id', 'categoria', 'precio', 'precio_mxn', 'precio_usd',
       'fecha_conversion', 'propiedad', 'metros_total', 'metros_construido',
       'precio_m2_terreno', 'precio_m2_construido', 'tiempo_de_publicacion',
       'meses_transcurridos', 'meses_transcurridos_fecha', 'tipo', 'Status',
       'estacionamientos', 'recamaras', 'banos', 'medio_banos', 'bano_total',
       'seguridad_privada', 'tipo_lugar', 'Colonia', 'Municipio', 'Estado',
       'CP', 'ubicacion', 'url', 'descripcion', 'Colonia_Estandarizada',
       'codigo_postal'],
      dtype='object')

In [31]:
#df_correctos[['ubicacion','Colonia','Municipio', 'Estado','tipo_lugar','Colonia_Estandarizada', 'codigo_postal']].to_csv('colonias_estandarizadas.csv', index=False, header=True, mode='w', encoding='utf-8')

In [32]:
# Identificar colonias estandarizadas que no se repiten o ya se encuentran estandarizadas en el diccionario
colonias_no_en_diccionario = df_correctos[df_correctos['Colonia_Estandarizada'].apply(lambda x: x not in diccionario)].copy()
# Crear un DataFrame de nuevas entradas sin duplicados, manteniendo las columnas del DataFrame original
nuevas_entradas = colonias_no_en_diccionario[['ubicacion','Colonia','Municipio', 'Estado','tipo_lugar','Colonia_Estandarizada', 'codigo_postal']].drop_duplicates()

# Actualizar el diccionario con las nuevas colonias sin reemplazar las existentes
# Excluir entradas vacías antes de la concatenación para evitar el FutureWarning
nuevas_entradas_sin_vacios = nuevas_entradas[nuevas_entradas['Colonia_Estandarizada'].notna()]
diccionario = pd.concat([diccionario, nuevas_entradas_sin_vacios]).drop_duplicates(keep='first')
diccionario

,ubicacion,Colonia,Municipio,Estado,tipo_lugar,Colonia_Estandarizada,codigo_postal
0,"San Baltasar Campeche, Puebla, Puebla",san baltasar campeche,puebla,puebla,NaN,san baltazar campeche,72550.0
1,"Bosques de San Sebastián, Puebla, Puebla",bosques de san sebastian,puebla,puebla,NaN,bosques san sebastian,72310.0
2,"Las Hadas Mundial 86, Puebla, Puebla",las hadas mundial 86,puebla,puebla,NaN,las hadas mundial 86,72070.0
4,"Plazas de Amalucan, Puebla, Puebla",plazas de amalucan,puebla,puebla,NaN,plazas amalucan,72310.0
5,"Héroes de Puebla, Puebla, Puebla",heroes de puebla,puebla,puebla,NaN,heroes de puebla,72520.0
...,...,...,...,...,...,...,...
8120,"Bellavista, Puebla, Puebla",bellavista,puebla,puebla,NaN,bella vista,72500.0
242,"Hacienda Del Camino Real, Puebla, Puebla",hacienda del camino real,puebla,puebla,None,camino real,72595.0
477,"Las 3 Cruces, Puebla, Puebla",las 3 cruces,puebla,puebla,None,tres cruces,72595.0
1005,"Agrícola Villa Albertina, Puebla, Puebla",agricola villa albertina,puebla,puebla,None,villa albertina,72495.0


In [33]:
# Exportar el diccionario actualizado al archivo original, pero sin sobrescribirlo
# En su lugar, agregar las nuevas entradas como nuevos registros
diccionario.to_csv('colonias_estandarizadas.csv', index=False, header=False, mode='a', encoding='utf-8')

ASIGNAR VALORES DE CODIGO POSTAL DIRECTOS

In [34]:
# Crear un diccionario para almacenar los códigos postales previos
codigos_postales_previos = {}
# Iterar sobre los registros sin código postal
for idx, row in sin_codigo_f.iterrows():
    # Verificar si ya existe un código postal previo para la colonia estandarizada actual
    colonia_estandarizada_actual = row['Colonia_Estandarizada']
    if colonia_estandarizada_actual in codigos_postales_previos:
        # Si ya existe, usar el valor previo
        new_codigo_postal = codigos_postales_previos[colonia_estandarizada_actual]
        print(f"Usando código postal previo para '{colonia_estandarizada_actual}': {new_codigo_postal}")
    else:
        # Solicitar un nuevo código postal si no existe en el diccionario
        new_codigo_postal = input(f"Introduce código postal para '{colonia_estandarizada_actual}': ")
        codigos_postales_previos[colonia_estandarizada_actual] = new_codigo_postal  # Guardar el nuevo código postal
    # Asignar el código postal al DataFrame
    sin_codigo_f.at[idx, 'codigo_postal'] = new_codigo_postal
    print("Cambio realizado por: ", new_codigo_postal)
    print("\n---\n")
print("Proceso completado.")

C:\Users\yoe11\AppData\Local\Temp\ipykernel_17044\337428235.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  sin_codigo_f.at[idx, 'codigo_postal'] = new_codigo_postal


Cambio realizado por:  0

---

Cambio realizado por:  72761

---

Usando código postal previo para 'villas atlixco': 72761
Cambio realizado por:  72761

---

Cambio realizado por:  72091

---

Cambio realizado por:  72761

---

Cambio realizado por:  0

---

Usando código postal previo para '': 0
Cambio realizado por:  0

---

Cambio realizado por:  73682

---

Usando código postal previo para 'centro': 72091
Cambio realizado por:  72091

---

Usando código postal previo para '': 0
Cambio realizado por:  0

---

Usando código postal previo para 'centro': 72091
Cambio realizado por:  72091

---

Usando código postal previo para '': 0
Cambio realizado por:  0

---

Usando código postal previo para '': 0
Cambio realizado por:  0

---

Usando código postal previo para 'centro': 72091
Cambio realizado por:  72091

---

Usando código postal previo para 'centro': 72091
Cambio realizado por:  72091

---

Usando código postal previo para 'snte': 0
Cambio realizado por:  0

---

Usando código po

In [35]:
df.shape

(3476, 32)

In [36]:
df_correctos = pd.concat([df_correctos,sin_codigo_f])
df_correctos.shape

(3476, 32)

In [37]:
sin_codigos = df_correctos[df_correctos['codigo_postal'].isna()]
sin_codigos

,id,categoria,precio,precio_mxn,precio_usd,fecha_conversion,propiedad,metros_total,metros_construido,precio_m2_terreno,...,tipo_lugar,Colonia,Municipio,Estado,CP,ubicacion,url,descripcion,Colonia_Estandarizada,codigo_postal


In [38]:
df = df_correctos

____
# 11. Variables oficiales

In [39]:
df = df.drop(['denominacion', 'precio_desc','baños_desc','baños_completos','medios_baños',], axis=1, errors='ignore') #'fecha_conversion'

In [ ]:
# Tiempo de publicación limpio:
df['tiempo_de_publicacion'] = df['fecha_estandarizada']

EXTRA

In [141]:
df['Status'] = 'Venta'
df['id'] = range(1, len(df) + 1)

ORDEN OFICIAL

In [ ]:
columnas_ordenadas = ['id', 'categoria', 'precio','precio_mxn','precio_usd','fecha_conversion', 'propiedad', 'metros_total', 'metros_construido',
                      'precio_m2_terreno', 'precio_m2_construido','tiempo_de_publicacion', 'meses_transcurridos','meses_transcurridos_fecha', 'tipo','Status',
                      'estacionamientos', 'recamaras', 'banos', 'medio_banos', 'bano_total','seguridad_privada', 'tipo_lugar','Colonia','Municipio','Estado','CP','ubicacion','url','descripcion']
df = df[columnas_ordenadas]

In [4]:
columnas_ordenadas = ['id', 'categoria', 'precio','propiedad', 'metros_total', 'metros_construido',
       'precio_m2_terreno', 'precio_m2_construido',
       'meses_transcurridos', 'tipo', 'Estado',
       'estacionamientos', 'recamaras', 'banos', 'medio_banos', 'bano_total',
       'seguridad_privada', 'Colonia', 'CP',  'url', ]
df = df[columnas_ordenadas]

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    132 non-null    int64  
 1   categoria             122 non-null    object 
 2   precio                132 non-null    int64  
 3   propiedad             132 non-null    object 
 4   metros_total          132 non-null    int64  
 5   metros_construido     132 non-null    int64  
 6   precio_m2_terreno     132 non-null    float64
 7   precio_m2_construido  132 non-null    float64
 8   meses_transcurridos   132 non-null    int64  
 9   tipo                  132 non-null    object 
 10  Estado                132 non-null    object 
 11  estacionamientos      132 non-null    int64  
 12  recamaras             132 non-null    int64  
 13  banos                 132 non-null    int64  
 14  medio_banos           132 non-null    int64  
 15  bano_total            1

# 13. Exportar

Cambia el nombre por el que desees en 'archivo'

In [8]:
df.isna().sum()

id                            0
categoria                    10
precio                        0
precio_mxn                    0
precio_usd                    0
fecha_conversion              0
propiedad                     0
metros_total                  0
metros_construido             0
precio_m2_terreno             0
precio_m2_construido          0
tiempo_de_publicacion         0
meses_transcurridos           0
meses_transcurridos_fecha     0
tipo                          0
Estado                        0
estacionamientos              0
recamaras                     0
banos                         0
medio_banos                   0
bano_total                    0
seguridad_privada             0
Colonia                       0
CP                            0
ubicacion                     0
url                           0
descripcion                   0
fecha_estandarizada           0
dtype: int64

In [9]:
df['categoria'].fillna('', inplace=True)

In [6]:
import os

archivo = 'abrill_2024'  # Nombre de archivo a exportar
ext = '.csv'  # Extensión del archivo
ciudad = 'tulancingo'  # Nombre de la ciudad en minúscula

ruta = os.path.join('..', 'ciudades', ciudad, 'clean')
if not os.path.exists(ruta):
    os.makedirs(ruta)

archivo_export = os.path.join(ruta, archivo + ext)

# Exportar el DataFrame a la ruta y archivo creados
df.to_csv(archivo_export, encoding='utf-8', index=False)
print(f"Archivo guardado en: {archivo_export}")

Archivo guardado en: ..\ciudades\tulancingo\clean\abrill_2024.csv


In [ ]:
import os

archivo = 'nombre_del_archivo'  # Nombre de archivo a exportar
ext = '.csv'  # Extensión del archivo
ciudad = ''  # Nombre de la ciudad en minúscula

ruta = os.path.join('..', 'ciudades', ciudad, 'clean')
if not os.path.exists(ruta):
    os.makedirs(ruta)

archivo_export = os.path.join(ruta, archivo + ext)

# Exportar el DataFrame a la ruta y archivo creados
df.to_csv(archivo_export, encoding='utf-8', index=False)
print(f"Archivo guardado en: {archivo_export}")

Archivo guardado en: ..\ciudades\puebla\clean\agosto_2024.csv
